In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mohinh/dulieu


In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier

In [3]:
data=pd.read_csv('/kaggle/input/mohinh/dulieu')

In [4]:
data

,year,category,topic,indicator,data_value,confidence_limit_low,confidence_limit_high
0,2011,0.0,2.0,10.0,36.370628,32.607573,40.178076
1,2011,0.0,2.0,10.0,36.370628,32.607573,40.178076
2,2011,0.0,2.0,10.0,36.370628,32.607573,40.178076
3,2011,0.0,2.0,10.0,36.370628,32.607573,40.178076
4,2011,0.0,2.0,10.0,36.370628,32.607573,40.178076
...,...,...,...,...,...,...,...
34999,2011,0.0,2.0,10.0,36.370628,32.607573,40.178076
35000,2011,0.0,2.0,10.0,36.370628,32.607573,40.178076
35001,2011,0.0,2.0,10.0,36.370628,32.607573,40.178076
35002,2011,0.0,2.0,10.0,36.370628,32.607573,40.178076


In [5]:
X= data.drop('topic', axis=1)

In [6]:
scaler = StandardScaler()
X_std = scaler.fit_transform(X)
X

,year,category,indicator,data_value,confidence_limit_low,confidence_limit_high
0,2011,0.0,10.0,36.370628,32.607573,40.178076
1,2011,0.0,10.0,36.370628,32.607573,40.178076
2,2011,0.0,10.0,36.370628,32.607573,40.178076
3,2011,0.0,10.0,36.370628,32.607573,40.178076
4,2011,0.0,10.0,36.370628,32.607573,40.178076
...,...,...,...,...,...,...
34999,2011,0.0,10.0,36.370628,32.607573,40.178076
35000,2011,0.0,10.0,36.370628,32.607573,40.178076
35001,2011,0.0,10.0,36.370628,32.607573,40.178076
35002,2011,0.0,10.0,36.370628,32.607573,40.178076


In [7]:
y = np.array(data['topic'])
y

array([2., 2., 2., ..., 2., 2., 2.])

In [35]:
ratio = 0.2 # huấn luyện, xác thực và kiểm tra theo tỉ lệ 70% − 10% − 20%
X_train,X_test,y_train,y_test = train_test_split(X_std, y, test_size= ratio, shuffle= True, stratify=y)

In [36]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size= 1/8, shuffle= True, stratify=y_train)

In [37]:
X_train.shape

(24502, 6)

In [38]:
X_test.shape

(7001, 6)

In [39]:
X_validation.shape

(3501, 6)

# 1. NaiveBayes

In [40]:
from sklearn.naive_bayes import  GaussianNB, MultinomialNB, CategoricalNB
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

In [41]:
model.score(X_validation, y_validation)


0.8066266780919737

In [42]:
model.score(X_test, y_test)

0.806313383802314

In [43]:
model_KNN_5 = KNeighborsClassifier(n_neighbors = 5)
model_KNN_7 = KNeighborsClassifier(n_neighbors = 7)
model_KNN_5.fit(X_train, y_train)
model_KNN_7.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [44]:
model_KNN_5.score(X_validation, y_validation) 

0.9948586118251928

In [45]:
model_KNN_7.score(X_test, y_test)

0.9892872446793315

# 2. Cross Valudation

In [46]:
from sklearn.model_selection import cross_val_score,RepeatedKFold, KFold, StratifiedKFold

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size= ratio, shuffle= True, stratify=y)

In [48]:
kf = KFold(n_splits=5)
kf.split(X_train, y_train)
train = kf.split(X_train, y_train)

In [49]:
fold_indices=[] 
for i, (train_index, test_index) in enumerate(train):
    fold_indices.append((train_index, test_index))

In [50]:
score_max = [0,0]
for k in range(3, 1000):
    model_KNN = KNeighborsClassifier(n_neighbors = k)
    score_k = cross_val_score(model_KNN,
                X_train,
                y_train,
                scoring ='accuracy',
                cv=fold_indices).mean()
    if score_k>score_max[0]:
        score_max[0]= score_k
        score_max[1]= k

In [51]:
score_max

[0.9953933034917235, 3]

In [52]:
model_KNN = KNeighborsClassifier(n_neighbors = score_max[1])

In [53]:
model_KNN.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [54]:
model_KNN.score(X_test, y_test)

0.9968575917726039